ML2 project----drug generation using LSTM by anku Rani

importing libraries and loading data

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('data.csv')

In [ ]:
df.head(4)

,zinc_id,smiles
0,ZINC000002040470,Cc1ccc2c(C)ccc(C(C)C)c2c1
1,ZINC000005963383,COc1c(C(C)C)cc2c(c1OC)[C@@]13CCCC(C)(C)[C@@H]1...
2,ZINC000002242633,CC1=C[C@H]2[C@H](CC1)C2(C)C
3,ZINC000005781270,Cc1ccc(C)c2occc12


In [ ]:
df.describe()

,zinc_id,smiles
count,101378,101378
unique,101378,101354
top,ZINC000085814951,COc1cc(/C=C/C(=O)OC[C@@H]2O[C@H](S/C(CC/C=C/[S...
freq,1,2


In [1]:
!pip install kora -q
import kora.install.rdkit

     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 


In [2]:
from rdkit import Chem
from numpy import random
import numpy as np

converting data into .txt file

In [ ]:

np.savetxt(r'/content/chembl_smiles.txt', df.smiles, fmt='%s')


converting it into npz format so that it can be processed

In [3]:

def load_chembl():
    f = open('/content/chembl_smiles.txt')
    lines = f.readlines()
    lines = [line.rstrip() for line in lines]
    f.close()
    return lines

chembl = load_chembl()
data_set = []
dictionary = {}
'''
create a character dictionary
'''
def added_to_dictionary(smile):
    for char in smile:
        if char not in dictionary:
            dictionary[char] = True

print('processing smiles !!!')

'''
! is the start token and ? is the end token
'''
for i in range(len(chembl)):
    if len(chembl[i])<=100:
        smile = '!'+chembl[i]+' '
        data_set.append(smile)
        added_to_dictionary(smile)

print('saving smiles')
vocabs = [ele for ele in dictionary]
print(len(vocabs))

'''
save the smiles string and vocabs
'''
np.savez_compressed('/content/smiles_data.npz', data_set=np.array(data_set, dtype=object),vocabs=np.array(vocabs), dtype=object)


processing smiles !!!
saving smiles
53


references:

https://ihaque.github.io/SIML/preprocessing.html
https://github.com/LamUong/Generate-novel-molecules-with-LSTM/blob/master/generative_model/data_loading.py



In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import time, math
from random import randint

def load_data():
    f = np.load('/content/smiles_data.npz',allow_pickle=True)
    return f['data_set'],f['vocabs']

def tensor_from_chars_list(chars_list,vocabs,cuda):
    tensor = torch.zeros(len(chars_list)).long()
    for c in range(len(chars_list)):
        tensor[c] = vocabs.index(chars_list[c])
    return tensor.view(1,-1)
'''
create batches of batch_size
'''
def process_batch(sequences,batch_size,vocabs,cuda):
    chunk_len = len(sequences[0])-1
    end_of_i = int(len(sequences)/batch_size)*batch_size
    batches=[]
    for i in range(0,len(sequences),batch_size):
        input_list = []
        output_list = []
        for j in range(i,i+batch_size,1):
            if j <len(sequences):
                input_list.append(tensor_from_chars_list(sequences[j][:-1],vocabs,cuda))
                output_list.append(tensor_from_chars_list(sequences[j][1:],vocabs,cuda))
        inp = Variable(torch.cat(input_list, 0))
        print('inp---',inp)
        target = Variable(torch.cat(output_list, 0))
        print('target----',target)
        if cuda:
            inp = inp.cuda()
            print('inp_cuda',inp)
            target = target.cuda()
        batches.append((inp,target))
        print('batches-------',batches)
        print('length of batches',len(batches))

    train_split = int(0.9*len(batches))
    print('train split----------',train_split)
    return batches[:train_split],batches[train_split:]

In [6]:
'''
group all the same length smiles together to process them in batch
'''
def process_data_to_batches(data,batch_size,vocabs,cuda):
    hash_length_data = {}
    for ele in data:
        l = len(ele)
        if l>=3:
            if l not in hash_length_data:
                hash_length_data[l] = []
            hash_length_data[l].append(ele)
    print("------------hash_length_data",hash_length_data)
    train_batches = []
    val_batches = []
    for length in hash_length_data:
        train,val = process_batch(hash_length_data[length],batch_size,vocabs,cuda)
        print('hash_length_data[length]',hash_length_data[length])
        print('batch_size',batch_size)
        print('vocabs',vocabs)
        print('cuda',cuda)
        train_batches.extend(train)
        val_batches.extend(val)
    return train_batches,val_batches

def get_random_batch(train_batches):
    print('length of train_batches',len(train_batches))
    r_n = randint(0,len(train_batches)-1)
    return train_batches[r_n]

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
#!pip install numpy==1.16.2
#from data_loading import *
'''
the model
'''
class generative_model(nn.Module):
    def __init__(self, vocabs_size, hidden_size, output_size, embedding_dimension, n_layers):
        super(generative_model, self).__init__()
        self.vocabs_size = vocabs_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding_dimension = embedding_dimension
        self.n_layers = n_layers

        self.embedding = nn.Embedding(vocabs_size, embedding_dimension)
        self.rnn = nn.LSTM(embedding_dimension, hidden_size, n_layers, dropout = 0.2)
        self.linear = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        batch_size = input.size(0)
        input = self.embedding(input)
        output, hidden = self.rnn(input.view(1, batch_size, -1), hidden)
        output = self.linear(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        hidden=(Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return hidden

data,vocabs=load_data()
print(data)
vocabs = list(vocabs)
vocabs_size = len(vocabs)
print(vocabs_size)
output_size = len(vocabs)
batch_size = 128
cuda = True
hidden_size = 1024
embedding_dimension =  248
n_layers=3
lr = 0.005
n_batches = 200000
print_every = 100
plot_every = 10
save_every = 1000
end_token = ' '

print("processing batches ...")
train_batches,val_batches = process_data_to_batches(data,batch_size,vocabs,cuda)
print('length of train batches',len(train_batches))
print('length of val batches', len(val_batches))
print("finish processing batches")


model = generative_model(vocabs_size,hidden_size,output_size,embedding_dimension,n_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

if cuda:
    model = model.cuda()
    criterion = criterion.cuda()

def propagation(inp, target, mode):
    batch_size = inp.size(0)
    sequence_length = inp.size(1)
    hidden = model.init_hidden(batch_size)
    if cuda:
        hidden = (hidden[0].cuda(), hidden[1].cuda())
    if mode=='train':
        model.zero_grad()
    loss = 0
    for c in range(sequence_length):
        output, hidden = model(inp[:,c], hidden)
        loss += criterion(output, target[:,c])
    if mode=='train':
        loss.backward()
        optimizer.step()
    return loss.data/sequence_length


def evaluate(prime_str='!', temperature=0.4):
    max_length = 200
    inp = Variable(tensor_from_chars_list(prime_str,vocabs,cuda)).cuda()
    batch_size = inp.size(0)
    hidden = model.init_hidden(batch_size)
    if cuda:
        hidden = (hidden[0].cuda(), hidden[1].cuda())
    predicted = prime_str
    while True:
        output, hidden = model(inp, hidden)
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        # Add predicted character to string and use as next input
        predicted_char = vocabs[top_i]

        if predicted_char ==end_token or len(predicted)>max_length:
            return predicted

        predicted += predicted_char
        inp = Variable(tensor_from_chars_list(predicted_char,vocabs,cuda)).cuda()
    return predicted

start = time.time()
all_losses = []
loss_avg = 0



['!Cc1cc(-c2csc(N=C(N)N)n2)cn1C '
 '!Brc1cccc(Nc2ncnc3ccncc23)c1NCCN1CCOCC1 '
 '!COc1c(O)cc(O)c(C(=N)Cc2ccc(O)cc2)c1O ' ...
 '!Nc1ncnc2c1ncn2C1CC(OP(=O)(O)O)C(COP(=O)(O)O)S1 '
 '!Nc1ncnc2c1ncn2C1CC(O)C(COP(=O)(O)O)[S+]1[O-] '
 '![N-]=[N+]=NC1C(COP(=O)(O)O)OC(n2cnc3c(N)ncnc32)C1OP(=O)(O)O ']
53
processing batches ...


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        [ 0,  1,  1,  ..., 10,  2,  3],
        ...,
        [ 0,  1,  1,  ...,  2,  2,  3],
        [ 0,  1,  1,  ...,  3,  9,  8],
        [ 0, 16,  2,  ..., 10,  2,  3]], device='cuda:0'), tensor([[16,  9,  1,  ...,  8,  3, 12],
        [ 1, 25,  2,  ...,  2,  3, 12],
        [ 1,  1,  1,  ...,  2,  3, 12],
        ...,
        [ 1,  1,  8,  ...,  2,  3, 12],
        [ 1,  1,  1,  ...,  9,  8, 12],
        [16,  2,  3,  ...,  2,  3, 12]], device='cuda:0')), (tensor([[ 0, 16,  9,  ..., 16, 10, 16],
        [ 0,  1, 11,  ...,  3,  9,  8],
        [ 0,  8,  1,  ...,  8, 10,  8],
        ...,
        [ 0, 16,  1,  ...,  1,  1,  6],
        [ 0,  1,  1,  ...,  6, 11,  3],
        [ 0,  8,  2,  ...,  2,  3,  8]], device='cuda:0'), tensor([[16,  9,  1,  ..., 10, 16, 12],
        [ 1, 11,  3,  ...,  9,  8, 12],
        [ 8,  1,  1,  ..., 10,  8, 12],
        ...,
        [16,  1,  3,  ...,  1,  6, 12],
        [ 1,  1, 16,  ..., 11,  3, 12]

In [ ]:
for batch in range(1,n_batches+1):
    inp, target = get_random_batch(train_batches)
    loss = propagation(inp, target, 'train')  
    loss_avg += loss
    if batch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), batch, batch / n_batches * 100, loss))
        print(evaluate('!'), '\n')
    if batch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0
    if batch %save_every ==0:
        torch.save(model.state_dict(), 'mytraining.pt')

Streaming output truncated to the last 5000 lines.
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 473
length of train_batches 4

testing and printing generated drugs

In [8]:
class generative_model(nn.Module):
    def __init__(self, vocabs_size, hidden_size, output_size, embedding_dimension, n_layers):
        super(generative_model, self).__init__()
        self.vocabs_size = vocabs_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding_dimension = embedding_dimension
        self.n_layers = n_layers

        self.embedding = nn.Embedding(vocabs_size, embedding_dimension)
        self.rnn = nn.LSTM(embedding_dimension, hidden_size, n_layers, dropout = 0.2)
        self.linear = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        batch_size = input.size(0)
        input = self.embedding(input)
        output, hidden = self.rnn(input.view(1, batch_size, -1), hidden)
        output = self.linear(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        hidden=(Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return hidden

In [9]:
data,vocabs=load_data()
data = set(list(data))
vocabs = list(vocabs)
vocabs_size = len(vocabs)
output_size = len(vocabs)
batch_size = 128
cuda = True
hidden_size = 1024
embedding_dimension =  248
n_layers=3
end_token = ' '

In [10]:
model = generative_model(vocabs_size,hidden_size,output_size,embedding_dimension,n_layers)
model.load_state_dict(torch.load('mytraining.pt'))
if cuda:
    model = model.cuda()
model.eval()

generative_model(
  (embedding): Embedding(53, 248)
  (rnn): LSTM(248, 1024, num_layers=3, dropout=0.2)
  (linear): Linear(in_features=1024, out_features=53, bias=True)
)

In [34]:
def evaluate(prime_str='!', temperature=0.4):
    max_length = 200
    inp = Variable(tensor_from_chars_list(prime_str,vocabs,cuda)).cuda()
    batch_size = inp.size(0)
    hidden = model.init_hidden(batch_size)
    if cuda:
        hidden = (hidden[0].cuda(), hidden[1].cuda())
    predicted = prime_str
    while True:
        output, hidden = model(inp, hidden)
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        # Add predicted character to string and use as next input
        predicted_char = vocabs[top_i]

        if predicted_char ==end_token or len(predicted)>max_length:
            return predicted

        predicted += predicted_char
        inp = Variable(tensor_from_chars_list(predicted_char,vocabs,cuda)).cuda()
    return predicted

code to print structure

In [38]:
print('generated structure 1', evaluate(prime_str='!', temperature=0.4))
print('generated structure 2', evaluate(prime_str='!', temperature=0.4))
print('generated structure 3', evaluate(prime_str='!', temperature=0.4))
print('generated structure 4', evaluate(prime_str='!', temperature=0.4))
print('generated structure 5', evaluate(prime_str='!', temperature=0.4))

generated structure 1 !CC(C)CC(NC(=O)C(CCCNC(N)=O)C(C)C)C(=O)O
generated structure 2 !CCc1ccc(C(=O)NCC2CCCCC2)cc1
generated structure 3 !O=C(O)c1ccc(NC(=O)CSc2nnc(-c3ccccc3)s2)cc1
generated structure 4 !CCOC(=O)C1=C(C)NC(=O)C(NC(=O)c2ccccc2)C1c1ccc(Cl)cc1
generated structure 5 !NC(=O)C(Cc1ccccc1)NC(=O)C(Cc1ccccc1)NC(=O)C1CCCCC1


#code to check if the structure is valid or not using chem

In [14]:
def valid_smile(smile):
    return Chem.MolFromSmiles(smile) is not None
def get_canonical_smile(smile):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smile))
def valid_smiles_at_temp(temp):
    range_test = 100
    c=0
    for i in range(range_test):
        s= evaluate(prime_str='!', temperature=temp)[1:] # remove the first character !.
        if valid_smile(s):
            print(s)
            c+=1
    return float(c)/range_test
def smiles_in_db(smile):
    smile = '!'+get_canonical_smile(smile)+' '
    if smile in data:
        return True
    return False

def percentage_variety_of_valid_at_temp(temp):
    range_test = 100
    c_v=0
    c_nd=0
    for i in range(range_test):
        s= evaluate(prime_str='!', temperature=temp)[1:] # remove the first character !.
        if valid_smile(s):
            c_v+=1
            if not smiles_in_db(s):
                c_nd+=1
    return float(c_nd)/c_v

In [41]:
valid_smile("CC(C)CC(NC(=O)C(CCCNC(N)=O)C(C)C)C(=O)O")

True

first generated structure is valid

In [49]:
get_canonical_smile('CC(C)CC(NC(=O)C(CCCNC(N)=O)C(C)C)C(=O)O')

'CC(C)CC(NC(=O)C(CCCNC(N)=O)C(C)C)C(=O)O'

In [50]:
smiles_in_db('CC(C)CC(NC(=O)C(CCCNC(N)=O)C(C)C)C(=O)O')

False

It is generated drug and not present in the database as well

checking 2nd generated structure

In [52]:
drug='CCc1ccc(C(=O)NCC2CCCCC2)cc1'
print('valid 2nd generated drug', valid_smile(drug))
print('2nd canonical structure', get_canonical_smile(drug))
print('checking if drug is present in database', smiles_in_db(drug))

valid 2nd generated drug True
2nd canonical structure CCc1ccc(C(=O)NCC2CCCCC2)cc1
checking if drug is present in database False


checking 3rd generated structure

In [56]:
drug_3='O=C(O)c1ccc(NC(=O)CSc2nnc(-c3ccccc3)s2)cc1'
print('valid 3rd generated drug', valid_smile(drug_3))
print('3rd canonical structure', get_canonical_smile(drug_3))
print('checking if drug is present in database', smiles_in_db(drug_3))

valid 3rd generated drug True
3rd canonical structure O=C(CSc1nnc(-c2ccccc2)s1)Nc1ccc(C(=O)O)cc1
checking if drug is present in database False


checking 4th generated structure

In [55]:
drug_4='CCOC(=O)C1=C(C)NC(=O)C(NC(=O)c2ccccc2)C1c1ccc(Cl)cc1'
print('valid 4th generated drug', valid_smile(drug_4))
print('4th canonical structure', get_canonical_smile(drug_4))
print('checking if drug is present in database', smiles_in_db(drug_4))

valid 4th generated drug True
4th canonical structure CCOC(=O)C1=C(C)NC(=O)C(NC(=O)c2ccccc2)C1c1ccc(Cl)cc1
checking if drug is present in database False


checking 5th generated structure

In [59]:
drug_5='NC(=O)C(Cc1ccccc1)NC(=O)C(Cc1ccccc1)NC(=O)C1CCCCC1'
print('valid 5th generated drug', valid_smile(drug_5))
print('5th canonical structure', get_canonical_smile(drug_5))
print('checking if drug is present in database', smiles_in_db(drug_5))

valid 5th generated drug True
5th canonical structure NC(=O)C(Cc1ccccc1)NC(=O)C(Cc1ccccc1)NC(=O)C1CCCCC1
checking if drug is present in database False
